In [1]:
from init import *
import time

In [2]:
model = Model()

In [3]:
m = mcsdk.examples.BuildModel_Simple()
m.evaluations.append(EvaluationPoint(0,2))
m.json()

'{"TimeStart": 0, "TimeSteps": 10, "NumPaths": 10000, "updaters": [{"name": "IndependentBrownianMotion"}, {"name": "BrownianMotion", "start": 0.1, "args": [0.2, 2]}], "evaluations": [{"state": 0, "time": 2}], "RandomSeed": -1}'

In [4]:
r = requests.post(f'{server}/model',m.json())
r.json()

{'mean': [0.5227665305137634],
 'names': ['BrownianMotion'],
 'npaths': [10000],
 'skewness': [0.029656510800123215],
 'stddev': [2.848874568939209],
 'time_points': [2.0],
 'time_steps': [10]}

In [5]:
m = mcsdk.examples.BuildModel_Simple()
m.NumPaths = 1000000
m.TimeSteps = 10000
for t in [1,2,5,10]:
    m.evaluations.append(EvaluationPoint(0,t))
time_start = time.time()
r = requests.post(f'{server}/model',m.json())
time_end = time.time()
r.json()

{'mean': [0.2981778681278229,
  0.503634512424469,
  1.108009934425354,
  2.1092159748077393],
 'names': ['BrownianMotion'],
 'npaths': [1000000, 1000000, 1000000, 1000000],
 'skewness': [-0.0012283348478376865,
  0.001641100156120956,
  0.004758447874337435,
  0.0035282562021166086],
 'stddev': [1.9996137619018555,
  2.8280715942382812,
  4.470082759857178,
  6.323413372039795],
 'time_points': [1.0, 2.0, 5.0, 10.0],
 'time_steps': [1000, 2000, 5000, 10000]}

In [6]:
print(f'{m.NumPaths*m.TimeSteps/(time_end-time_start)*1e-9:.1f} nanoseconds to make a single time step')

3.0 nanoseconds to make a single time step


# Two processes, Three evaluations

In [7]:
model = mcsdk.examples.TwoProcessesModel()
model.TimeSteps = 10000
for time in [1,2,5]:
    model.evaluations.append(EvaluationPoint(state=0,time=time))
r = requests.post(f'{server}/model',model.json())
r.json()

{'mean': [0.2912197411060333,
  0.30108997225761414,
  0.45512402057647705,
  0.5199389457702637,
  1.0351451635360718,
  1.1028811931610107],
 'names': ['BrownianMotion', 'BrownianMotion'],
 'npaths': [10000, 10000, 10000, 10000, 10000, 10000],
 'skewness': [0.01221475936472416,
  0.0034566200338304043,
  -0.016960060223937035,
  -0.0011860266095027328,
  -0.03123420663177967,
  -0.012307627126574516],
 'stddev': [1.994964361190796,
  1.9942896366119385,
  2.843177556991577,
  2.8572192192077637,
  4.488191604614258,
  4.473225116729736],
 'time_points': [1.0, 2.0, 5.0],
 'time_steps': [2000, 4000, 10000]}

In [8]:
er = EvaluationResults(r.json(),model)
er

2 states with 3 evaluations

In [9]:
er.df()

,name,state,time,step,npaths,mean,mean_error,stddev,skewness
0,BrownianMotion,0,1.0,2000,10000,0.291220,0.019950,1.994964,0.012215
1,BrownianMotion,1,1.0,2000,10000,0.301090,0.019943,1.994290,0.003457
2,BrownianMotion,0,2.0,4000,10000,0.455124,0.028432,2.843178,-0.016960
3,BrownianMotion,1,2.0,4000,10000,0.519939,0.028572,2.857219,-0.001186
4,BrownianMotion,0,5.0,10000,10000,1.035145,0.044882,4.488192,-0.031234
5,BrownianMotion,1,5.0,10000,10000,1.102881,0.044732,4.473225,-0.012308
